In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from bayesian_torch.layers.variational_layers.conv_variational import Conv2dReparameterization, Conv2dReparameterization_Multivariate
from torch.distributions import LowRankMultivariateNormal, kl_divergence
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from models import resnet20_multi
from bayesian_torch.models.bayesian.resnet_variational import resnet20
from bayesian_torch.layers.variational_layers.conv_variational import Conv2dReparameterization, Conv2dReparameterization_Multivariate
import torch
import numpy as np
import torch.nn.functional as F
import copy
import torch.nn as nn
from distill import get_conv_layers

from bayesian_torch.models.deterministic.resnet import resnet20 as resnet20_deterministic
dnn = resnet20_deterministic()
dnn.load_state_dict(torch.load('runs/cifar/resnet20/reference/dnn/bs128_lr0.001_mc100_temp_1.0_ep300_kd_False_alpha_0.0_moped_False_20240820-012216/best_model.pth'))
dnn.eval().cuda()
from utils import get_dataset

class opt:
    
    bs = 1
    data = 'cifar'
    multi_gpu = False
    
args = opt()
train_loader, test_loader = get_dataset(args)

in planes is reduced by 1 times since the computation is heavy
CIFAR-10 dataset is loaded
Files already downloaded and verified


In [2]:
dnn_conv_layers = get_conv_layers(dnn)

In [3]:
import os
import torch
from torch.utils.data import Dataset, DataLoader

class ActivationDataset(Dataset):
    def __init__(self, folder_path):
        self.input_dir = os.path.join(folder_path, 'input')
        self.output_dir = os.path.join(folder_path, 'output')
        
        # input과 output 디렉토리 안의 파일 목록을 가져옵니다.
        self.input_files = sorted([f for f in os.listdir(self.input_dir) if f.endswith('.pt')])
        self.output_files = sorted([f for f in os.listdir(self.output_dir) if f.endswith('.pt')])
        
        # input과 output 파일 수가 같은지 확인
        assert len(self.input_files) == len(self.output_files), "Input and output file counts do not match."

    def __len__(self):
        return len(self.input_files)
    
    def __getitem__(self, idx):
        # input과 output 파일 경로
        input_path = os.path.join(self.input_dir, self.input_files[idx])
        output_path = os.path.join(self.output_dir, self.output_files[idx])
        
        # 텐서 로드
        input_tensor = torch.load(input_path)
        output_tensor = torch.load(output_path)
        
        return input_tensor.squeeze(), output_tensor.squeeze()


In [ ]:

for layer_idx, layer in enumerate(dnn_conv_layers):
    bnn = Conv2dReparameterization_Multivariate(
        in_channels=layer.in_channels,
        out_channels=layer.out_channels,
        kernel_size=layer.kernel_size[0],
        stride=layer.stride,
        padding=layer.padding,
        dilation=layer.dilation,
        groups=layer.groups,
        bias=False,
        rank = 'full'        
    )
    folder_path = f'./resent20_dnn_activations/layer_{layer_idx}'
    
    # Dataset과 DataLoader 생성
    dataset = ActivationDataset(folder_path)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

    # DataLoader를 통해 데이터 확인
    optimizer = torch.optim.Adam(bnn.parameters(), lr=1e-3)
    bnn = bnn.cuda().train()
    print(bnn.L_param.shape)
    for e in range(10):
        
        pbar = tqdm(dataloader, total=len(dataloader), desc=f"Epoch {e+1}")
        losses = []
        for inputs, outputs in pbar:
            inputs, outputs = inputs.cuda(), outputs.cuda()
            output_bnn , _ = bnn(inputs)
            
            loss = F.mse_loss(output_bnn, outputs, reduction='mean')
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            losses.append(loss.item())
            pbar.set_description(f"loss: {np.mean(losses):.4f}")
            
    optimizer = torch.optim.LBFGS(bnn.parameters())
    
    def closure():
        optimizer.zero_grad()
        output_bnn , _ = bnn(inputs)
        loss = F.mse_loss(output_bnn, outputs, reduction='mean')
        loss.backward()
        return loss
    
    optimizer.step(closure)
    print(f"Final loss: {loss.item()}")
    
    torch.save(bnn.state_dict(), f'./resent20_dnn_activations/layer_{layer_idx}/bnn_conv.pth')
    print(f"Model is saved at ./resent20_dnn_activations/layer_{layer_idx}/bnn_conv.pth")
    # break
    


torch.Size([432, 432])


loss: 56.9411: 100%|██████████| 50000/50000 [04:48<00:00, 173.03it/s]


Final loss: 44.20319366455078
Model is saved at ./resent20_dnn_activations/layer_0/bnn_conv.pth
torch.Size([2304, 2304])


loss: 3661.1139: 100%|██████████| 50000/50000 [08:01<00:00, 103.84it/s]


Final loss: 5574.46875
Model is saved at ./resent20_dnn_activations/layer_1/bnn_conv.pth
torch.Size([2304, 2304])


...

loss: 1590601.7388: 100%|██████████| 50000/50000 [1:41:52<00:00,  8.18it/s]


Final loss: 969103.75
Model is saved at ./resent20_dnn_activations/layer_9/bnn_conv.pth
torch.Size([9216, 9216])


loss: 71873.9916: 100%|██████████| 50000/50000 [1:48:43<00:00,  7.66it/s]


Final loss: 107688.640625
Model is saved at ./resent20_dnn_activations/layer_10/bnn_conv.pth
torch.Size([9216, 9216])


loss: 1318677.7827:  92%|█████████▏| 46088/50000 [1:36:25<08:45,  7.44it/s]

In [13]:
def visualize_covariance(layer, i, save = False, file_name = None):
    
    mu = layer.mu_kernel.view(-1)
    L, D = layer.get_covariance_param()
    
    mvn = LowRankMultivariateNormal(mu, L, D)
    
    # plt.imshow(mvn.covariance_matrix.cpu().detach().numpy()[:9*i, :9*i])
    # plt.colorbar(fraction=0.046, pad=0.04)
    
    if save:
        torch.save(layer.state_dict(), file_name)
    
    return mvn

In [ ]:
mvn_list = []

for layer_idx, layer in tqdm(enumerate(dnn_conv_layers), total = len(dnn_conv_layers)):
    bnn_test = Conv2dReparameterization_Multivariate(
        in_channels=layer.in_channels,
        out_channels=layer.out_channels,
        kernel_size=layer.kernel_size[0],
        stride=layer.stride,
        padding=layer.padding,
        dilation=layer.dilation,
        groups=layer.groups,
        bias=False,
        rank = 'full'        
    )
    folder_path = f'./resent20_dnn_activations/layer_{layer_idx}'
    
    bnn_test.load_state_dict(torch.load(f'./resent20_dnn_activations/layer_{layer_idx}/bnn_conv.pth'))
    
    mvn_list.append(visualize_covariance(bnn_test, 5, save = False, file_name = f'./resent20_dnn_activations/layer_{layer_idx}/bnn_conv_cov.pth'))
    
    

In [ ]:
plt.figure(figsize=(100, 100))
for idx in range(len(mvn_list)):
    i=6
    plt.subplot(len(mvn_list), 1, idx+1)
    plt.imshow(mvn_list[idx].covariance_matrix.cpu().detach().numpy()[:9*i, :9*i])
    plt.colorbar()